# Segmentace obrazu - text a jeho čtení
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu.

### Import knihoven a konfigurace

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import cv2
from pypylon import pylon 
from IPython.display import clear_output, display

%matplotlib inline

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Funkce k připojení kamery.

In [ ]:
CAM_WIDTH = 1080
CAM_HEIGHT = 720

# List of features to create widgets
FEATURES = [
    {
        "name": "GainRaw",
        "type": "int"
    },
    {
        "name": "ExposureTimeRaw",
        "type": "int",
        "max": 35000,
        "min": 35,
        "step": 140,
        "value": 444
    },
    {
        "name": "Height",
        "type": "int_text",
        "max": 1200,
        "min": 100,
        "step": "5"
    },
    {
        "name": "Width",
        "type": "int_text",
        "max": 1920,
        "min": 100,
        "step": "5"
    }
]

def connect_camera(serial_number):
    ''' Connects camera specified with its serial number
    
    Parameters
    ----------
    serial_number : string
        Camera's serial number.
    grabbed_images_path : string
        Path to folder where the saved images will be stored.
    Returns
    -------
    camera : object
    '''
    info = None
    for i in pylon.TlFactory.GetInstance().EnumerateDevices():
        if i.GetSerialNumber() == serial_number:
            info = i
            break
    else:
        print('Camera with {} serial number not found'.format(serial_number))

    # VERY IMPORTANT STEP! To use Basler PyPylon OpenCV viewer you have to call .Open() method on you camera
    if info is not None:
        camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateDevice(info)) 
        camera.Open()
        return camera
    else:
        return None    

Funkce pro zobrazení okna s přidáním eventu na klik myší.

In [ ]:
def print_xy(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        print('x = %d, y = %d'% (x, y))      
        
def show_camera_window(img, scale=1):
    window_name = 'Camera capture (' + serial_number + ')'
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, int(img.shape[1] * scale), int(img.shape[0] * scale))
    cv2.setMouseCallback(window_name, print_xy)
    cv2.imshow(window_name, img)

Metody předzpracování.

In [ ]:
def to_gray(img):
    ''' Converts image to monochrome
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    if len(img.shape) == 2:
        return img
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def to_hsv(img):
    ''' Converts image to HSV (hue, saturation, value) color space.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return dst

def negative(img):
    ''' Converts image to its negative
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    dst = 255 - img
    return dst

def crop(img, tl_x, tl_y, br_x, br_y):
    ''' Crops image by added coordinates
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    tl_x : int
        TOP-LEFT corner's x-coordinate
    tl_y : int
        TOP-LEFT corner's y-coordinate
    br_x : int
        BOTTOM-RIGHT corner's x-coordinate
    br_y : int
        BOTTOM-RIGHT corner's y-coordinate
    Returns
    -------
    Ouput image.
    '''
    roi = img[tl_y:br_y, tl_x:br_x]
    return roi    

Metody segmentace.

In [ ]:
def segmentation_one_threshold(img, threshold):
    '''Segments image into black & white using one threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    threshold : int
        Pixels with value lower than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    _, dst = cv2.threshold(img, threshold, 255, cv2.THRESH_BINARY)
    return dst

def segmentation_auto_threshold(img):
    '''Segments image into black & white using automatic threshold
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    Returns
    -------
    Ouput image.
    '''
    _, dst = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return dst

def segmentation_two_thresholds(img, lower, higher):
    '''Segments image into black & white using two thresholds
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    lower : int
        Pixels with value lower than threshold are considered black, the others white.
    higher : int
        Pixels with value higher than threshold are considered black, the others white.
    Returns
    -------
    Ouput image.
    '''
    return cv2.inRange(img, min(lower, higher), max(lower, higher))

def segmentation_adaptive_threshold(img, size, constant=0):
    '''Segments image into black & white using calculated adaptive 
    threshold using Gaussian function in pixel neighbourhood.
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    size : int
        Size of used gaussian. Lowest value is 3. Algorithm uses only odd numbers.
    constant : int
        Value that is added to calculated threshlod. It could be negative as well as zero as well as positive number.
    Returns
    -------
    Ouput binary image.
    '''
    if size < 3:
        size = 3
    elif size % 2 == 0:
        size -= 1
    return cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, size, int(constant))

def contours(img_bin, area=0):
    '''Finds contours in binary image and filters them using their area. Then it draws binary image
    from filtered contours. It counts contours as well.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    area : int
        Size of contour that is used to filter all smaller contours out.
    Returns
    -------
    contour_drawn : numpy.ndarray
        Output binary image with drawn filled filtered contours.
    count : int
        Number of found and filtered contours.
    '''
    _, contours, _  = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours =  [c for c in contours if cv2.contourArea(c) > area]
    contour_drawn = cv2.drawContours(np.zeros(img_bin.shape, dtype=np.uint8), contours, -1, color=(255, 255, 255), thickness=cv2.FILLED)
    return contour_drawn, len(contours)    

Metody filtrace.

In [ ]:
def filtration_median(img, filter_size):
    '''Filters image noise using median algorithm
    
    Parameters
    ----------
    img : numpy.ndarray
        Input image.
    filter_size : int
        Size of median filter.
    Returns
    -------
    Ouput image.
    '''
    return cv2.medianBlur(img, filter_size)   

def fill_holes(img_bin, close=False, size=5):
    '''Fill holes in found contours. It could merge the contour using close input with appropriate size.
    
    Parameters
    ----------
    img_bin : numpy.ndarray
        Input binary image.
    close : boolean
        If it should merge contours with missing points using close operation.
    size : int
        Size of close operation element.
    Returns
    -------
    Ouput binary image.
    '''
    if close:
        struct = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
        img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, struct)
    res, _ = contours(img_bin)
    return res

Metoda pro zobrazení různého množství obrázků.

In [ ]:
def show_images(*imgs, scale=1, window_name='Image preview'):
    """ Opens multiple image previews depending on the length of the input *imgs list.
    The preview is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    *imgs : list
        Multiple input images which have to be shown.
    scale : double
        Scale of shown image window.
    window_name : Optional[string]
        An optional window name.
    Returns
    -------
    None
    """
    for i, img in enumerate(imgs, 1):
        window_name_id = window_name + ' ' + str(i)
        cv2.namedWindow(window_name_id, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
        cv2.resizeWindow(window_name_id, img.shape[1] * scale, img.shape[0] * scale)
        cv2.moveWindow(window_name_id, 0, i*10)

    while 1:
        for i, img in enumerate(imgs, 1):
            cv2.imshow(window_name + ' ' + str(i), img)
            
        k = cv2.waitKey(0)

        if k == ord('q'):
            break

    cv2.destroyAllWindows()

Metoda pro OCR obrazu

In [ ]:
def ocr(img):
    """Detects text in the file."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient.from_service_account_json('text-recognition-key.json')

    import io
    import os
    path = 'D:\\temp\\'
    os.makedirs(path)
    path += 'img_ocr.png'
    cv2.imwrite(path, img)
    
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    if os.path.exists(path):
        os.remove(path)
    return texts[1].description

Metody ostatní

In [ ]:
def to_intensity(hue_angle):
    '''Converts color angle in HUE definition into intensity value of brightness image in opencv
    
    Parameters
    ----------
    hue_angle : int
        Angle in HUE definition.
    Returns
    -------
    Integer value that represents the same HUE value but in opencv brightness image.
    '''
    return int(hue_angle * 0.5)

---

Seznam funkcí pro přehlednost:
- `connect_camera(...)`
- `show_images(...)`


- `to_gray(...)`
- `to_hsv(...)`
- `negative(...)`
- `crop(...)`


- `segmentation_one_threshold(...)`
- `segmentation_auto_threshold(...)`
- `segmentation_two_thresholds(...)`
- `segmentation_adaptive_threshold(...)`
- `contours(...)`


- `filtration_median(...)`
- `fill_holes(...)`


- `ocr(...)`


- `to_intensity(...)`

### Úkol
Zvolte vhodné funkce pro segmentaci obrazu a přečtěte text na obrazu snímaném kamerou.

Pro testování algoritmu využijte funkci `algorithm()`. Funkce je vložena do snímání kamery. Je tak možné v reálném čase pozorovat, co se s obrazem děje. Po změně obsahu funkce je však potřeba načíst znovu i blok pro spuštění snímání z kamery.

Po vyladění algoritmu, využijte obrázek z kamery a nechte ho přečíst OCR. V případě, že výsledek bude po porovnání `True`, úkol jste splnili.

#### 1) Nastavte správně kameru.

In [ ]:
serial_number = '' ### vlastní sériové číslo kamery
width = 1280
height = 720

camera = ...(serial_number) ###

#### 2) Doplňte funkční verze algoritmů pro nasvícení jednotlivými typy světel.

**2a)** Funkce pro segmentaci referenčního obrazce (černo-bílý). Osvětlení žádné navíc (při nízkých světelných podmínkách lze využít světlo DOME).

In [ ]:
def algorithm_reference(img):
    ###
   
    return ... ###

**2b)** Funkce pro segmentaci obrazce s využitím osvětlení pomocí dvou plošných světel typu bar.

In [ ]:
def algorithm_two_bars(img):
    ###
    
    return ... ###

**2c)** Funkce pro segmentaci obrazce s využitím osvětlení typu dark field (konzole se 4 bar světly).

In [ ]:
def algorithm_console(img):
    ###
    
    return ... ###

Definuje obsah funkce pro zpracování obrazu, konkrétně jeho segmentaci. Je třeba měnit právě upravovaný algoritmus.

In [ ]:
def impro(img):
    res = ...(img) ###

    show_camera_window(res, 1)
    return res

#### 3) Spusťte snímání a testujte algoritmus pro zpracování obrazu.

Info k oknu:
- Pro spuštění okna kamery je zapotřebí kliknout na tlačítko `Run Interact`. 
- Okno kamery se vypne stisknutím tlačítka `q`.
- Obrázek se uloží stisknutím tlačítka `s`.
- Po kliku levého tlačítka myši se vytisknou souřadnice místa v obraze.
- Pro projevení změn v nastavení pomocí GUI prvků je nutné ukončit okno kamery a znovu ho spustit.

In [ ]:
from pypylon_opencv_viewer import BaslerOpenCVViewer
    
viewer = BaslerOpenCVViewer(camera)
viewer.set_features(FEATURES)
viewer.set_impro_function(impro)
viewer.run_interaction_continuous_shot(window_size=(width, height))

#### 4) Využijte funkční algoritmus na uložené obrázky z kamery.
**POZOR:** Dokud nebude vstupní obraz vypadat následovně, **NEPOUŠTĚJTE** se do další části.

<img src="fit_preprocessed.png" width="200">

In [ ]:
# Celkem 3 obrázky
image_paths = [
    ### 
]

# Automatický preprocessing vytvořenými algoritmy
images_preprocessed = [
    algorithm_reference(cv2.imread(image_paths[0])),
    algorithm_two_bars(cv2.imread(image_paths[1])),
    algorithm_console(cv2.imread(image_paths[2]))
]

#### 5) Využijte OCR na výsledný binární obrázek
**POZOR:** Existuje limit pro použití OCR API. Může se tedy stát, že bude vyhazovat chybu. Je třeba počkat. Proto se doporučuje poslední část cvičení dělat postupně po obrázcích.

Deklarace výsledného pole textů ze všech úloh

In [ ]:
texts = ['', '', '']

Testovací blok pro OCR pro obrázek 1

In [ ]:
texts[0] = ocr(images_preprocessed[0])
print('Přečtený text je: ' + texts[0])

Testovací blok pro OCR pro obrázek 2

In [ ]:
texts[1] = ocr(images_preprocessed[1])
print('Přečtený text je: ' + texts[1])

Testovací blok pro OCR pro obrázek 2

In [ ]:
texts[2] = ocr(images_preprocessed[2])
print('Přečtený text je: ' + texts[2])

#### 6) Zkontrolujte správnost

In [ ]:
ref_text = 'FIT'

for i, text in enumerate(texts):
    print('Přečtený text je: ' + text)
    
    if text == ref_text:
        print('-> Úkol ' + str(i+1) + ' jste splnili!')
    else:
        print('-> Úkol je třeba dál ladit ...')    
    print('')